# 0. Librerías

Cargamos las librerias necesarias para ejecutar el cuaderno.

In [1]:
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff

# 1. Carga de Datos: Exploración del alquiler de bicicletas

Exploramos el dataset de Bikeshare para entender cómo el clima, la estacionalidad y los días festivos afectan la demanda de bicicletas.

Este dataset viene de serie con cierto tratamiento de los datos (ej.: la normalización de las temperaturas, humedad, velocidad del viento...)

In [2]:
# Carga del dataset por hora
df = pd.read_csv("../data/hour.csv")
# Leemos las primeras entradas del dataset
df.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


### Descripción de las columnas del dataset

El archivo `/data/readme.txt` aporta información sobre las columnas presentes en el dataset.

- **instant**: índice del registro
- **dteday**: fecha
- **season**: estación (1: primavera, 2: verano, 3: otoño, 4: invierno)
- **yr**: año (0: 2011, 1: 2012)
- **mnth**: mes (1 a 12)
- **hr**: hora (0 a 23)
- **holiday**: si el día es festivo (1) o no (0)
- **weekday**: día de la semana (0: domingo, 6: sábado)
- **workingday**: si es día laboral (1) o no (0)
- **weathersit**: situación climática (1: Despejado, 2: Niebla/Nubes, 3: Lluvia ligera/Nieve, 4: Lluvia intensa/Nieve)
- **temp**: temperatura normalizada (dividida por el valor max 41)
- **atemp**: sensación térmica normalizada (dividida por el valor max 50)
- **hum**: humedad normalizada (dividida por 100, tanto por 1)
- **windspeed**: velocidad del viento normalizada (dividida por el valor max 67)
- **casual**: número de usuarios ocasionales
- **registered**: número de usuarios registrados
- **cnt**: total de alquileres de bicicletas (casual + registrados)


Puesto que el dataset contiene varias columnas con valores normalizados y nuestro objetivo en este notebook es realizar una exploración descriptiva (no preparar los datos para un modelo predictivo concreto), vamos a realizar los siguientes pasos para facilitar la interpretación de los resultados:

- Renombrar las columnas para que sean más comprensibles.
- Desnormalizar las columnas que contienen valores escalados, devolviéndolas a sus unidades originales.
- Crear un índice temporal a partir de la fecha y la hora.
- Eliminar columnas que no son necesarias para el análisis exploratorio.

Estas transformaciones permitirán interpretar los datos de manera más clara y realizar visualizaciones más informativas.

In [3]:
# Renombramos columnas para facilitar su comprensión
df = df.rename(columns={
    "instant": "instante",
    "dteday": "fecha",
    "season": "estacion",
    "yr": "año",
    "mnth": "mes",
    "hr": "hora",
    "holiday": "festivo",
    "weekday": "dia_semana",
    "workingday": "dia_laboral",
    "weathersit": "sit_meteorologica",
    "hum": "norm_hum",
    "temp": "norm_temp",
    "atemp": "norm_atemp",
    "windspeed": "norm_windspeed",
    "casual": "casual",
    "registered": "registrado",
    "cnt": "num_usuarios"
})

In [4]:
# Desnormalizamos las columnas previamente normalizadas
df["temperatura"] = (df["norm_temp"] * 41).round(1) 
df["temperatura_sensacion"] = (df["norm_atemp"] * 50).round(1)
df["humedad"] = (df["norm_hum"] * 100).round(2)
df["velocidad_viento"] = (df["norm_windspeed"] * 67).round(2)

In [5]:
# Convertimos la columna 'dteday' a tipo datetime
df["timestamp"] = pd.to_datetime(df["fecha"]) + pd.to_timedelta(df["hora"], unit="h")
df["timestamp"] = df["timestamp"].dt.strftime("%Y-%m-%d %H:%M")
df = df.set_index("timestamp")

In [6]:
# Eliminamos columnas innecesarias por ahora
df = df.drop(columns=["instante", "año", "mes", "hora", "fecha", "norm_temp", "norm_atemp", "norm_hum", "norm_windspeed"], axis=1)

## Descripción del dataset

Una vez seleccionadas las columnas necesarias para el análisis, vamos a ver una descripción de los valores que presentan cada una.

In [9]:
df.describe()

,estacion,festivo,dia_semana,dia_laboral,sit_meteorologica,casual,registrado,num_usuarios,temperatura,temperatura_sensacion,humedad,velocidad_viento
count,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000
mean,2.501640,0.028770,3.003683,0.682721,1.425283,35.676218,153.786869,189.463088,20.374636,23.791783,62.722884,12.736233
std,1.106918,0.167165,2.005771,0.465431,0.639357,49.305030,151.357286,181.387599,7.894645,8.589473,19.292983,8.196891
min,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.800000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,1.000000,0.000000,1.000000,4.000000,34.000000,40.000000,13.900000,16.700000,48.000000,7.000000
50%,3.000000,0.000000,3.000000,1.000000,1.000000,17.000000,115.000000,142.000000,20.500000,24.200000,63.000000,13.000000
75%,3.000000,0.000000,5.000000,1.000000,2.000000,48.000000,220.000000,281.000000,27.100000,31.100000,78.000000,17.000000
max,4.000000,1.000000,6.000000,1.000000,4.000000,367.000000,886.000000,977.000000,41.000000,50.000000,100.000000,57.000000


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17379 entries, 2011-01-01 00:00 to 2012-12-31 23:00
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   estacion               17379 non-null  int64  
 1   festivo                17379 non-null  int64  
 2   dia_semana             17379 non-null  int64  
 3   dia_laboral            17379 non-null  int64  
 4   sit_meteorologica      17379 non-null  int64  
 5   casual                 17379 non-null  int64  
 6   registrado             17379 non-null  int64  
 7   num_usuarios           17379 non-null  int64  
 8   temperatura            17379 non-null  float64
 9   temperatura_sensacion  17379 non-null  float64
 10  humedad                17379 non-null  float64
 11  velocidad_viento       17379 non-null  float64
dtypes: float64(4), int64(8)
memory usage: 1.7+ MB


In [11]:
df.nunique()

estacion                   4
festivo                    2
dia_semana                 7
dia_laboral                2
sit_meteorologica          4
casual                   322
registrado               776
num_usuarios             869
temperatura               50
temperatura_sensacion     65
humedad                   89
velocidad_viento          30
dtype: int64

In [8]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

## Distribución de la demanda de bicicletas

In [ ]:
fig = px.histogram(df, x='cnt', nbins=30, title='Distribución de la demanda de bicicletas')
fig.show()

## Relación entre variables climáticas y la demanda

In [ ]:
fig1 = px.scatter(df, x='temp', y='cnt', title='Temperatura vs Alquileres')
fig2 = px.scatter(df, x='hum', y='cnt', title='Humedad vs Alquileres')
fig3 = px.scatter(df, x='windspeed', y='cnt', title='Viento vs Alquileres')
fig1.show()
fig2.show()
fig3.show()

## Efecto del día de la semana, estación y feriados

In [ ]:
fig = px.box(df, x='weekday', y='cnt', title='Alquileres por día de la semana')
fig.show()
fig = px.box(df, x='season', y='cnt', title='Alquileres por estación')
fig.show()
fig = px.box(df, x='holiday', y='cnt', title='Alquileres en días festivos vs normales')
fig.show()

## Matriz de correlación entre variables

In [ ]:

corrs = df[['temp', 'atemp', 'hum', 'windspeed', 'cnt']].corr().round(2)
fig = ff.create_annotated_heatmap(
    z=corrs.values,
    x=list(corrs.columns),
    y=list(corrs.index),
    annotation_text=corrs.values,
    colorscale='Viridis')
fig.update_layout(title='Matriz de correlación')
fig.show()